In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import seaborn as sns

# 1. READING THE CSV FILE
def load_data(file_path):
    """
    Load data from CSV and perform initial exploration
    """
    print("Loading dataset...")
    df = pd.read_csv(file_path)

    # Basic information about the dataset
    print(f"Dataset shape: {df.shape}")
    print(f"Number of samples: {df.shape[0]}")
    print(f"Number of features: {df.shape[1]}")

    # Check for missing values
    missing_values = df.isnull().sum().sum()
    print(f"Missing values: {missing_values}")

    # Extract class distribution
    if 'group' in df.columns:
        print("Class distribution:")
        print(df['group'].value_counts())

    return df

# 2. FEATURE TRANSFORMATIONS
def transform_features(X):
    """
    Apply feature transformations to improve separability
    """
    # Create a copy to avoid modifying the original
    X_transformed = X.copy()

    # 1. Log transform for highly skewed features
    # First find highly skewed columns (absolute skew > 1)
    skewed_features = []
    for col in X_transformed.columns:
        if abs(X_transformed[col].skew()) > 1:
            skewed_features.append(col)

    # Apply log transform to skewed features (adding small constant to handle zeros)
    for col in skewed_features:
        if (X_transformed[col] >= 0).all():  # Only apply to non-negative columns
            X_transformed[col] = np.log1p(X_transformed[col])

    print(f"Applied log transform to {len(skewed_features)} skewed features")

    # 2. Feature interactions (for selected most important features)
    # Since we don't know which features are most important yet,
    # we'll create interactions between features with high variance

    # Get top 10 features with highest variance
    variances = X_transformed.var().sort_values(ascending=False)
    top_features = variances.index[:10]

    # Create pairwise interactions between top features
    for i, feat1 in enumerate(top_features):
        for feat2 in top_features[i+1:]:
            interaction_name = f"{feat1}_x_{feat2}"
            X_transformed[interaction_name] = X_transformed[feat1] * X_transformed[feat2]

    print(f"Added {len(top_features) * (len(top_features) - 1) // 2} interaction features")

    # 3. Polynomial features for top features (squared terms)
    for feat in top_features:
        X_transformed[f"{feat}_squared"] = X_transformed[feat] ** 2

    print(f"Added {len(top_features)} polynomial features")

    # 4. Remove features with very low variance
    # Calculate variance for each feature
    variances = X_transformed.var()
    # Remove features with variance close to 0
    low_var_threshold = 1e-5
    low_var_features = variances[variances < low_var_threshold].index
    X_transformed = X_transformed.drop(columns=low_var_features)

    print(f"Removed {len(low_var_features)} features with near-zero variance")

    # 5. Detect and handle outliers
    # Using z-score method: replace extreme values (|z| > 3) with winsorized values
    for col in X_transformed.columns:
        z_scores = (X_transformed[col] - X_transformed[col].mean()) / X_transformed[col].std()
        outliers = (abs(z_scores) > 3)
        if outliers.sum() > 0:
            # Replace outliers with threshold values (winsorizing)
            upper_limit = X_transformed[col].mean() + 3 * X_transformed[col].std()
            lower_limit = X_transformed[col].mean() - 3 * X_transformed[col].std()
            X_transformed.loc[z_scores > 3, col] = upper_limit
            X_transformed.loc[z_scores < -3, col] = lower_limit

    print("Handled outliers using winsorization")

    return X_transformed

# 3. MY PCA IMPLEMENTATION
class MyPCA:
    def __init__(self, n_components):
        """
        Initialize PCA with number of components
        """
        self.n_components = n_components
        self.components = None
        self.mean = None
        self.explained_variance = None
        self.explained_variance_ratio = None
        self.cumulative_explained_variance_ratio = None

    def fit(self, X):
        """
        Fit PCA model to data X
        """
        # Center the data (subtract mean)
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean

        # Compute covariance matrix
        cov_matrix = np.cov(X_centered, rowvar=False)

        # Compute eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # Sort eigenvalues and eigenvectors in descending order
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]

        # Store the top n_components eigenvectors
        self.components = eigenvectors[:, :self.n_components]

        # Calculate explained variance and ratios
        total_var = np.sum(eigenvalues)
        self.explained_variance = eigenvalues[:self.n_components]
        self.explained_variance_ratio = self.explained_variance / total_var
        self.cumulative_explained_variance_ratio = np.cumsum(self.explained_variance_ratio)

        return self

    def transform(self, X):
        """
        Transform X using the PCA components
        """
        # Center the data using saved mean
        X_centered = X - self.mean

        # Project data onto principal components
        X_transformed = np.dot(X_centered, self.components)

        return X_transformed

    def fit_transform(self, X):
        """
        Fit to data then transform it
        """
        self.fit(X)
        return self.transform(X)

# 4. MAIN EXECUTION FLOW
def main(file_path, test_n_components=None):
    """
    Main function to execute the workflow
    """
    # Load the dataset
    df = load_data(file_path)

    # Separate features and target
    y = df['group'].copy()
    X = df.drop('group', axis=1)
    X = X.select_dtypes(include=[np.number])  # Keep only numerical features

    print(f"\nOriginal features shape: {X.shape}")

    # Apply feature transformations
    X_transformed = transform_features(X)
    print(f"Transformed features shape: {X_transformed.shape}")

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_transformed)

    # If no specific n_components to test, we'll find optimal
    if test_n_components is None:
        # Test different numbers of components
        test_n_components = [2, 5, 10, 15, 20, 30, 50, 75, 100]

    results = []

    for n_components in test_n_components:
        print(f"\nTesting with {n_components} PCA components:")

        # Apply PCA
        pca = MyPCA(n_components=n_components)
        X_pca = pca.fit_transform(X_scaled)

        # Print explained variance
        print(f"Explained variance ratio: {pca.explained_variance_ratio}")
        print(f"Cumulative explained variance: {pca.cumulative_explained_variance_ratio[-1]:.4f}")

        # Apply KMeans clustering
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(X_pca)

        # Convert class labels to binary (0, 1)
        # Assuming 'Cancer' should be 1 and 'Normal' is 0
        y_true = (y == 'Cancer').astype(int)

        # We need to check if cluster labels match actual labels or are flipped
        # Calculate accuracy for original and flipped labels
        acc_original = accuracy_score(y_true, clusters)
        acc_flipped = accuracy_score(y_true, 1 - clusters)

        # Use the mapping that gives higher accuracy
        if acc_original >= acc_flipped:
            y_pred = clusters
            accuracy = acc_original
        else:
            y_pred = 1 - clusters
            accuracy = acc_flipped

        # Calculate metrics
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        silhouette = silhouette_score(X_pca, clusters)

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Silhouette Score: {silhouette:.4f}")

        # Store results
        results.append({
            'n_components': n_components,
            'explained_variance': pca.cumulative_explained_variance_ratio[-1],
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'silhouette': silhouette
        })

        # Plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        print("Confusion Matrix:")
        print(cm)

    # Convert results to dataframe
    results_df = pd.DataFrame(results)

    # Find best n_components based on accuracy
    best_idx = results_df['accuracy'].idxmax()
    best_n_components = results_df.loc[best_idx, 'n_components']
    best_accuracy = results_df.loc[best_idx, 'accuracy']

    print(f"\nBest n_components: {best_n_components} with accuracy: {best_accuracy:.4f}")

    # Plot results
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    plt.plot(results_df['n_components'], results_df['explained_variance'], marker='o')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('Explained Variance vs. Components')

    plt.subplot(2, 2, 2)
    plt.plot(results_df['n_components'], results_df['accuracy'], marker='o')
    plt.xlabel('Number of Components')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. Components')

    plt.subplot(2, 2, 3)
    plt.plot(results_df['n_components'], results_df['f1'], marker='o')
    plt.xlabel('Number of Components')
    plt.ylabel('F1 Score')
    plt.title('F1 Score vs. Components')

    plt.subplot(2, 2, 4)
    plt.plot(results_df['n_components'], results_df['silhouette'], marker='o')
    plt.xlabel('Number of Components')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score vs. Components')

    plt.tight_layout()
    plt.savefig('pca_results.png')

    # Now let's visualize the clustering with the best number of components
    pca = MyPCA(n_components=best_n_components)
    X_pca = pca.fit_transform(X_scaled)

    # For visualization, we'll use only the first 2 components
    X_pca_2d = X_pca[:, :2]

    # Apply KMeans
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_pca)

    # Check if we need to flip cluster labels
    y_true = (y == 'Cancer').astype(int)
    acc_original = accuracy_score(y_true, clusters)
    acc_flipped = accuracy_score(y_true, 1 - clusters)

    if acc_original >= acc_flipped:
        y_pred = clusters
    else:
        y_pred = 1 - clusters

    # Plot the clusters
    plt.figure(figsize=(12, 6))

    # Plot based on predicted clusters
    plt.subplot(1, 2, 1)
    scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y_pred, cmap='viridis', alpha=0.6)
    plt.colorbar(scatter)
    plt.title('Clustering Results')
    plt.xlabel('PC1')
    plt.ylabel('PC2')

    # Plot based on true labels
    plt.subplot(1, 2, 2)
    scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y_true, cmap='viridis', alpha=0.6)
    plt.colorbar(scatter)
    plt.title('True Labels')
    plt.xlabel('PC1')
    plt.ylabel('PC2')

    plt.tight_layout()
    plt.savefig('clustering_visualization.png')

    return results_df

In [ ]:
# Example usage
# results = main("path_to_your_dataset.csv")

# For testing with specific number of components:
results = main("../Dataset/ABIDE2.csv", test_n_components=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150])